In [53]:
# The code was removed by DSX for sharing.

In [59]:
import pandas as pd
df = (df.withColumn('Age',df['Age'].cast('double'))
            .withColumn('SibSp',df['SibSp'].cast('double'))
            .withColumn('Parch',df['Parch'].cast('double'))
            .withColumn('Fare',df['Fare'].cast('double'))
            .withColumn('Survived',df['Survived'].cast('double')))

In [60]:
df.count()

891

In [84]:
numVars = ['Survived','Age','SibSp','Parch','Fare']
def countNull(df,var):
    return df.where(df[var].isNull()).count()
 
missing = {var: countNull(df,var) for var in numVars}
age_mean = df.groupBy().mean('Age').first()[0]
fare_mean = df.groupBy().mean('Fare').first()[0]
df = df.na.fill({'Age':age_mean, 'Fare':fare_mean, 'Survived':0, 'SibSp': 0, 'Parch': 0, 'Pclass': 'Null', 'Sex': 'Null', 'Ticket': 'Null', 'Embarked': 'Null'})

In [85]:
df.count()

891

In [86]:
df = df.drop('Cabin', 'PassengerId')

In [87]:
df.count()

891

In [88]:
df = df.na.drop()

891

In [89]:
df.count()

891

In [90]:
df.printSchema()

root
 |-- Survived: double (nullable = false)
 |-- Pclass: string (nullable = false)
 |-- Sex: string (nullable = false)
 |-- Age: double (nullable = false)
 |-- SibSp: double (nullable = false)
 |-- Parch: double (nullable = false)
 |-- Ticket: string (nullable = false)
 |-- Fare: double (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Title: string (nullable = true)



In [91]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
 
## created user defined function to extract title
getTitle = udf(lambda name: name.split('.')[0].strip(),StringType())
df = df.withColumn('Title', getTitle(df['Name']))
df = df.drop('Name')

AnalysisException: u'Cannot resolve column name "Name" among (Survived, Pclass, Sex, Age, SibSp, Parch, Ticket, Fare, Embarked, Title);'

In [92]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["Title","Pclass", "Sex", "Ticket", "Embarked"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer,encoder]

In [93]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "Survived", outputCol = "label")
stages += [label_stringIdx]

In [94]:
# Transform all features into a vector using VectorAssembler
numericCols = ["Age", "SibSp","Parch", "Fare"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [95]:
df.printSchema()

root
 |-- Survived: double (nullable = false)
 |-- Pclass: string (nullable = false)
 |-- Sex: string (nullable = false)
 |-- Age: double (nullable = false)
 |-- SibSp: double (nullable = false)
 |-- Parch: double (nullable = false)
 |-- Ticket: string (nullable = false)
 |-- Fare: double (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Title: string (nullable = true)



In [96]:
cols = df.columns
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(df)
dataset = pipelineModel.transform(df)

In [97]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
dataset.toPandas()

,label,features,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,22.000000,1,0,A/5 21171,7.2500,S,"Braund, Mr"
1,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,1,female,38.000000,1,0,PC 17599,71.2833,C,"Cumings, Mrs"
2,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,3,female,26.000000,0,0,STON/O2. 3101282,7.9250,S,"Heikkinen, Miss"
3,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,1,female,35.000000,1,0,113803,53.1000,S,"Futrelle, Mrs"
4,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,35.000000,0,0,373450,8.0500,S,"Allen, Mr"
5,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,29.699118,0,0,330877,8.4583,Q,"Moran, Mr"
6,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,1,male,54.000000,0,0,17463,51.8625,S,"McCarthy, Mr"
7,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,3,male,2.000000,3,1,349909,21.0750,S,"Palsson, Master"
8,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,3,female,27.000000,0,2,347742,11.1333,S,"Johnson, Mrs"
9,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,2,female,14.000000,1,0,237736,30.0708,C,"Nasser, Mrs"


In [98]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print trainingData.count()
print testData.count()

614
277


In [ ]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)
predictions.printSchema()

In [100]:
# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. For example's sake we will choose age & occupation
selected = predictions.select("label", "prediction", "probability", "Age")
selected.toPandas()

,label,prediction,probability,Age
0,0,0,"[0.999613489583, 0.000386510417427]",2.000000
1,0,0,"[0.999644110298, 0.000355889701929]",8.000000
2,0,0,"[0.999657951865, 0.00034204813521]",20.000000
3,0,0,"[0.998436179683, 0.00156382031652]",28.000000
4,0,0,"[0.999347061027, 0.000652938973184]",29.699118
5,0,0,"[0.998628063677, 0.00137193632339]",41.000000
6,0,0,"[0.998336286645, 0.00166371335485]",26.000000
7,0,0,"[0.997900287164, 0.00209971283563]",19.000000
8,0,0,"[0.970373984457, 0.0296260155427]",56.000000
9,0,0,"[0.989810861576, 0.0101891384241]",29.699118


In [103]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.8261634199134199

In [104]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [144]:
# Use test set here so we can measure the accuracy of our model on new data

predictions = cvModel.transform(testData)
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.8411525974025977

In [ ]:

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [105]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(trainingData)

In [106]:
print "numNodes = ", dtModel.numNodes
print "depth = ", dtModel.depth

numNodes =  15
depth =  3


In [108]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [109]:
selected = predictions.select("label", "prediction", "probability", "Age")
selected.show()

+-----+----------+--------------------+-------------+
|label|prediction|         probability|          Age|
+-----+----------+--------------------+-------------+
|  0.0|       0.0|[0.89473684210526...|          2.0|
|  0.0|       0.0|[0.89473684210526...|          8.0|
|  0.0|       0.0|[0.90977443609022...|         20.0|
|  0.0|       0.0|[0.90977443609022...|         28.0|
|  0.0|       0.0|[0.64864864864864...|29.6991176471|
|  0.0|       0.0|[0.90977443609022...|         41.0|
|  0.0|       0.0|[0.90977443609022...|         26.0|
|  0.0|       0.0|[0.90977443609022...|         19.0|
|  0.0|       0.0|[0.61458333333333...|         56.0|
|  0.0|       0.0|[0.90977443609022...|29.6991176471|
|  0.0|       0.0|[0.89473684210526...|          9.0|
|  0.0|       0.0|[0.89473684210526...|         11.0|
|  0.0|       0.0|[0.89473684210526...|29.6991176471|
|  0.0|       0.0|[0.90977443609022...|         22.0|
|  0.0|       0.0|[0.90977443609022...|         33.0|
|  0.0|       0.0|[0.9097744

In [110]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.8105248917748916

In [147]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1,2,6,10])
             .addGrid(dt.maxBins, [20,40,80])
             .build())

In [148]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [149]:
predictions = cvModel.transform(testData)

In [150]:
evaluator.evaluate(predictions)

0.8286796536796538

In [111]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [ ]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [112]:
predictions = rfModel.transform(testData)

In [113]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age")
selected.show()

+-----+----------+--------------------+-------------+
|label|prediction|         probability|          Age|
+-----+----------+--------------------+-------------+
|  0.0|       0.0|[0.58874019640607...|          2.0|
|  0.0|       0.0|[0.63262317512948...|          8.0|
|  0.0|       0.0|[0.745650043772,0...|         20.0|
|  0.0|       0.0|[0.73875796953007...|         28.0|
|  0.0|       0.0|[0.54303780107835...|29.6991176471|
|  0.0|       0.0|[0.73875796953007...|         41.0|
|  0.0|       0.0|[0.73875796953007...|         26.0|
|  0.0|       0.0|[0.745650043772,0...|         19.0|
|  0.0|       0.0|[0.58090732758633...|         56.0|
|  0.0|       0.0|[0.61407854340958...|29.6991176471|
|  0.0|       0.0|[0.62941686660526...|          9.0|
|  0.0|       0.0|[0.66083221052060...|         11.0|
|  0.0|       0.0|[0.66083221052060...|29.6991176471|
|  0.0|       0.0|[0.745650043772,0...|         22.0|
|  0.0|       0.0|[0.745650043772,0...|         33.0|
|  0.0|       0.0|[0.6802982

In [114]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.8195616883116882

In [152]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [153]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [154]:
predictions = cvModel.transform(testData)

In [155]:
evaluator.evaluate(predictions)

0.8270833333333332

In [156]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", weightCol='Survived')

# train the model
model = nb.fit(trainingData)

In [157]:
predictions = model.transform(testData)

In [159]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Fare")
selected.show()

+-----+----------+--------------------+-------+
|label|prediction|         probability|   Fare|
+-----+----------+--------------------+-------+
|  0.0|       1.0|[1.84504312137990...| 29.125|
|  0.0|       1.0|[1.31611270530997...| 29.125|
|  0.0|       1.0|[7.94509507056686...| 9.8458|
|  0.0|       1.0|[1.25071302379326...|  7.925|
|  0.0|       1.0|[2.15211063696365...|25.4667|
|  0.0|       1.0|[1.29784348506983...|14.1083|
|  0.0|       1.0|[4.15126982470329...| 7.8542|
|  0.0|       1.0|[8.93302492659600...|    0.0|
|  0.0|       1.0|[3.82696437112595...|30.6958|
|  0.0|       1.0|[5.48903364609699...|   26.0|
|  0.0|       1.0|[1.35752051933497...|   46.9|
|  0.0|       1.0|[5.63000252710607...|   46.9|
|  0.0|       1.0|[4.98268484065482...|  69.55|
|  0.0|       1.0|[3.05735634798281...| 7.7958|
|  0.0|       1.0|[2.42249562090087...| 8.6542|
|  0.0|       1.0|[1.06605970718637...| 7.2292|
|  0.0|       1.0|[3.24379940207277...|39.6875|
|  0.0|       1.0|[9.27820258059291...| 

In [160]:
evaluator.evaluate(predictions)

0.46298701298701317